In [4]:
from data_processing.preprocess_encoder_data import filter_and_dedup_encoder_data

filter_and_dedup_encoder_data(
    encoder_csv_path='data_processing/raw/encoder_data.csv',
    stroke_map_ty_path='data_processing/mapping/stroke_mapping_name.csv',
    hit_area_map_path='data_processing/mapping/hit_area_mapping.csv',
    stroke_map_llm_path='data_processing/mapping/stroke_mapping_llm.csv',
    output_csv_path='data_processing/processed/filtered_encoder_data.csv',
    dedup_keep='last',       
    sort_by=None             
)

done data_processing/processed/filtered_encoder_data.csv


In [5]:
from dataset_pipeline.video_cropper import VideoCropper

cropper = VideoCropper(output_path='Output',video_dir='Dataset/Video')
cropper.crop_videos('data_processing/processed/filtered_encoder_data.csv')

Cropping videos:   0%|          | 26/15838 [00:05<57:23,  4.59it/s]  


KeyboardInterrupt: 

In [ ]:
import os
from dataset_pipeline.caption_generator import CaptionGenerator
csv_path = 'data_processing/processed/filtered_encoder_data.csv'
caption_output_dir = 'generated_labels/caption'
os.makedirs(caption_output_dir, exist_ok=True)

caption_gen = CaptionGenerator(caption_output_dir)
caption_df = caption_gen.generate_captions(csv_path)
caption_csv = os.path.join(caption_output_dir, 'dataset_labels_caption.csv')
caption_df.to_csv(caption_csv, index=False)
print(f'Captions saved to: {caption_csv}')


Generating captions: 100%|██████████| 15838/15838 [00:00<00:00, 34973.53it/s]


Caption Json generated successfully.


In [ ]:
from dataset_pipeline.qa_generator import QAGenerator
csv_path = 'data_processing/processed/filtered_encoder_data.csv'
caption_output_dir = 'generated_labels/caption'
qa_output_dir = 'generated_labels/QA'
os.makedirs(qa_output_dir, exist_ok=True)
qa_output_path = os.path.join(qa_output_dir, 'qa_dataset.json')

templates_all = ["When does the {player} hits a {stroke} {hit_area}?"]
templates_ps  = ["When does the {player} hits a {stroke}?"]
templates_s   = ["When is a {stroke} hits?"]
templates_h   = ["Which stroke is hit {hit_area}?"]

qa_gen = QAGenerator(
    templates_with_all=templates_all,
    templates_player_stroke=templates_ps,
    templates_stroke_only=templates_s,
    templates_hit_area_only=templates_h,
    stroke_chunk_size=5,
    caption_csv_path=caption_csv
)
qa_gen.generate_by_rally(
    csv_path=csv_path,
    output_path=qa_output_path,
    num_questions_per_rally=14,
    val_to_train_ratio=0.05
)
print(f'QA dataset saved to: {qa_output_path}')


Generating QA by stroke-chunk: 100%|██████████| 1667/1667 [00:11<00:00, 139.43it/s]


Split complete: Total 54138, Train 44123, Val 10015, Moved 527
